In [1]:
from google.colab import drive

# ✅ 挂载 Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-
"""
feature_detector.py

端到端脚本：
1) 从 JSON 文件加载正负采样的 token-level 熵序列 和 错误/修正位置信息；
2) 提取时域 & 频域统计特征；
3) 拼接成分类特征向量并打标签（负样本=1，正样本=0）；
4) 训练 LogisticRegression 二分类器并评估性能。
"""

import os
import json
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# ---------- 配置项，请修改为你本地的实际路径 ----------
BASE_PATH      = "/content/drive/MyDrive/Cluster-proj"
start_index    = 700
end_index      = 731
range_tag      = f"{start_index}-{end_index}"
LOGITS_PATH    = f"{BASE_PATH}/output/llm_steps/whole_logits/deepseek7b-gsm-{range_tag}.json"
INDEX_PATH     = f"{BASE_PATH}/output/error_fix_index/deepseek-7b-{range_tag}_error_fix_index.json"

# ---------- 核心工具函数 ----------



In [7]:
# 1. 加载 JSON
with open(LOGITS_PATH, 'r') as f:
    logits_data = json.load(f)
with open(INDEX_PATH, 'r') as f:
    index_data  = json.load(f)

In [8]:
# -*- coding: utf-8 -*-
"""
feature_detector.py

端到端脚本：
1) 从 JSON 文件加载正负采样的 token-level 熵序列 和 错误/修正位置信息；
2) 只使用 sampling* 字段及其 correct_sampling_id 构建正负样本对；
3) 提取时域 & 频域统计特征；
4) 拼接成分类特征向量并打标签（负样本=1，正样本=0）；
5) 训练 LogisticRegression 二分类器并评估性能。
"""

import os
import json
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# ---------- 配置（修改为本地路径） ----------
BASE_PATH   = "/content/drive/MyDrive/Cluster-proj"
start_index = 700
end_index   = 731
range_tag   = f"{start_index}-{end_index}"
LOGITS_PATH = f"{BASE_PATH}/output/llm_steps/whole_logits/deepseek7b-gsm-{range_tag}.json"
INDEX_PATH  = f"{BASE_PATH}/output/error_fix_index/deepseek-7b-{range_tag}_error_fix_index.json"

# ---------- 特征提取函数 ----------
def extract_time_features(entropy):
    feats = {
        'mean': np.mean(entropy),
        'var': np.var(entropy),
        'skew': skew(entropy),
        'kurtosis': kurtosis(entropy),
        'max': np.max(entropy),
        'min': np.min(entropy)
    }
    peaks, props = find_peaks(entropy, height=None)
    feats['peak_count']    = len(peaks)
    feats['peak_prom_avg'] = float(np.mean(props['prominences'])) if 'prominences' in props else 0.0
    feats['autocorr1']     = float(np.corrcoef(entropy[:-1], entropy[1:])[0,1]) if len(entropy)>1 else 0.0
    return feats

def extract_freq_features(entropy):
    N  = len(entropy)
    yf = fft(entropy)
    xf = fftfreq(N, d=1)[:N//2]
    amp = np.abs(yf)[:N//2]
    power = amp**2
    total = np.sum(power) + 1e-8

    feats = {}
    feats['spec_centroid']      = float(np.sum(xf * amp) / (np.sum(amp)+1e-8))
    feats['band_mid_high_ratio']= float(np.sum(power[xf>0.1]) / total)
    geo_mean = np.exp(np.mean(np.log(power+1e-8)))
    feats['spec_flatness']      = float(geo_mean / (np.mean(power)+1e-8))
    csum = np.cumsum(power)
    idx  = np.where(csum >= 0.85*total)[0][0]
    feats['spec_rolloff']       = float(xf[idx])
    p_norm = power/total
    feats['spec_entropy']       = float(-np.sum(p_norm * np.log2(p_norm+1e-8)))
    return feats

# ---------- 主流程 ----------
def main():


    # 2. 构建 sampling 对
    paired = []
    for qid, meta_dict in index_data.items():
        if qid not in logits_data:
            continue
        for neg_sid, meta in meta_dict.items():
            # 只处理 sampling 开头的负样本
            if not neg_sid.startswith("sampling"):
                continue
            pos_sid = meta.get("correct_sampling_id")
            # 只有当正样本 ID 在 logits_data 中存在时才配对
            if pos_sid in logits_data[qid]:
                paired.append((qid, neg_sid, pos_sid))
            else:
                print(f"⚠️ 跳过 {qid} | {neg_sid} vs {pos_sid} — 不存在正确采样")

    # 3. 特征提取
    records = []
    for qid, neg_sid, pos_sid in paired:
        try:
            neg_probs = logits_data[qid][neg_sid]["token_probs"]
            pos_probs = logits_data[qid][pos_sid]["token_probs"]
            neg_seq = [tok["topk_info"]["entropy"] for tok in neg_probs]
            pos_seq = [tok["topk_info"]["entropy"] for tok in pos_probs]
        except Exception as e:
            print(f"跳过 {qid} | {neg_sid} vs {pos_sid} — {e}")
            continue

        if len(neg_seq)<4 or len(pos_seq)<4:
            continue

        neg_feats = {f"neg_{k}":v for k,v in {**extract_time_features(neg_seq),  **extract_freq_features(neg_seq)}.items()}
        pos_feats = {f"pos_{k}":v for k,v in {**extract_time_features(pos_seq),  **extract_freq_features(pos_seq)}.items()}

        # 负样本=1，正样本=0
        records.append({**neg_feats, **pos_feats, "label":1})
        records.append({**pos_feats, **neg_feats, "label":0})

    df = pd.DataFrame(records)
    X  = df.drop(columns="label")
    y  = df["label"]

    # 4. 划分训练/测试
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # 5. 标准化
    scaler = StandardScaler().fit(X_tr)
    X_tr_s = scaler.transform(X_tr)
    X_te_s = scaler.transform(X_te)

    # 6. 训练分类器
    clf = LogisticRegression(max_iter=1000, class_weight='balanced')
    clf.fit(X_tr_s, y_tr)

    # 7. 评估
    y_pred = clf.predict(X_te_s)
    y_prob = clf.predict_proba(X_te_s)[:,1]

    print("=== Classification Report ===")
    print(classification_report(y_te, y_pred, digits=4))
    print("=== ROC AUC Score ===")
    print(f"{roc_auc_score(y_te, y_prob):.4f}")

if __name__ == "__main__":
    main()


=== Classification Report ===
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         6
           1     0.2500    0.4000    0.3077         5

    accuracy                         0.1818        11
   macro avg     0.1250    0.2000    0.1538        11
weighted avg     0.1136    0.1818    0.1399        11

=== ROC AUC Score ===
0.1333
